<a href="https://colab.research.google.com/github/teerasitk/01205415/blob/main/TechnicalTradingWorkshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install
1. yfinance for accessing yahoo finance data
2. tensortrade for trade simulation

In [1]:
!pip install yfinance
!pip install tensortrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.6/32.6 MB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.1 MB/s eta 0:00:00
  Created wheel for tensortrade: filename=tensortrade-1.0.3-py3-none-any.whl size=134844 sha256=b60b10a9cf33c1515dceeded8bb6b229a455fdb218edafc58ff4b7e3eaf55565
  Stored in directory: /root/.cache/pip/wheels/7f/a7/75/792c00b8bcd136448db22cd2ca4b4bf0856a58a7dea881bcfc
Successfully built tensortrade


# Install Python Libraries

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensortrade.env.default as default
from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.instruments import Instrument
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.oms.exchanges import Exchange, ExchangeOptions
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.env.default.renderers import PlotlyTradingChart, FileLogger

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


# Download AOT from 2015 to 2023

In [3]:
data_frame = yf.download("AOT.BK", "2015-01-01", "2023-12-31")
data_frame.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-05,28.0,28.000000,27.200001,27.299999,24.920395,24970000
2015-01-06,26.9,27.400000,26.700001,27.200001,24.829109,24309000
2015-01-07,27.5,28.299999,27.500000,28.299999,25.833229,20682000
2015-01-08,28.9,29.000000,28.600000,28.900000,26.380930,23039000
2015-01-09,29.1,29.299999,28.900000,29.100000,26.563499,34221000


# Build Trade Environment

In [4]:
THB = Instrument(symbol="THB", precision=2, name="Thai Baht")
AOT = Instrument(symbol="AOT", precision=2, name="Airports of Thailand")
set_market = Exchange("set", service=execute_order,
                      options=ExchangeOptions(commission=0.0001))
# build exchange market
# commission is to 0.01%
buy_sell_prices = data_frame['Close'].tolist() # buy and sell prices are close prices
stream_prices = Stream.source(buy_sell_prices, dtype='float').rename("THB-AOT")
set_market = set_market(stream_prices

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Build Features

In [8]:
close_prices = data_frame.Close
high_prices = data_frame.High
low_prices = data_frame.Low

In [12]:
hist_len = 90 # look back 1 quarter
streams1 = [Stream.source(
    close_prices.shift(c)).rename(f"close_lag{c:02d}")
    for c in range(1, hist_len+1)] # 0-89: historical close prices
streams2 = [Stream.source(
    high_prices.shift(c)).rename(f"high_lag{c:02d}")
    for c in range(1, hist_len+1)] # 90-179: historical high prices
streams3 = [Stream.source(
    low_prices.shift(c)).rename(f"low_lag{c:02d}")
    for c in range(1, hist_len+1)] # 190-239: historical low prices
streams = streams1 + streams2 + streams3
streams.append(Stream.source(high_prices, # today max
                         dtype="float").rename("max_of_day"))
streams.append(Stream.source(low_prices, # today min
                         dtype="float").rename("min_of_day"))


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Build Data feed

In [ ]:
feed = DataFeed(streams)
# 0-89: historical close prices
# 90-179: historical high prices
# 190-239: historical low prices
# 240: max of the day
# 241: min of the day

# Initialize Portfolios

In [14]:
cash = Wallet(set_market, 100_000 * THB) # initialize at 100k THB
asset = Wallet(set_market, 0 * AOT)
portfolio = Portfolio(THB, [cash,
                            asset])

# Build Trade Environment

In [15]:
env = default.create(
        portfolio=portfolio, # start with our portfolio
        action_scheme=default.actions.BSH(cash=cash, asset=asset), # all in and out
        # I made some changes here
        reward_scheme=default.rewards.SimpleProfit(), # simple profit (to be used later)
        feed=feed, # data feed for historical price
        window_size=1, # historical data that we need
        min_periods=hist_len # starting trading location
        )

# Test the envornment

In [16]:
obs = env.reset()
print(obs)

[[29.9 28.6 28.8 28.7 28.9 28.8 28.9 28.8 28.8 29.2 29.  29.  28.9 29.1
  29.2 29.5 29.1 29.1 29.2 29.4 29.2 30.  29.3 29.3 29.2 29.1 29.2 29.3
  29.1 28.  28.  29.3 29.5 30.1 29.8 30.3 29.7 29.9 29.8 29.1 29.4 29.5
  29.6 29.7 29.5 29.7 30.1 29.3 29.7 30.3 30.9 31.4 30.4 30.6 30.7 31.1
  30.7 31.3 30.7 31.2 31.2 31.6 31.9 31.5 31.4 32.  31.6 31.4 32.2 32.4
  32.4 30.9 31.1 31.3 31.4 31.  29.5 29.  29.  29.  28.9 28.9 28.4 29.
  29.1 29.1 28.9 28.3 27.2 27.3 29.9 29.2 29.1 29.2 28.9 29.  29.4 29.
  29.2 29.2 29.  29.  29.1 29.2 29.5 29.6 29.4 29.4 29.6 29.6 30.  30.1
  29.3 29.5 29.6 29.5 29.4 29.7 29.3 28.3 29.6 29.6 30.1 30.2 30.4 30.4
  30.2 30.4 30.  29.5 29.6 29.9 29.8 29.8 30.  30.2 30.2 29.7 30.2 30.8
  31.4 31.4 30.9 31.1 31.2 31.3 31.4 31.3 31.3 31.5 31.7 31.9 32.1 31.8
  32.  32.2 32.  32.5 33.3 32.4 32.4 31.2 31.7 31.7 32.  31.1 29.5 29.2
  29.2 29.2 29.  29.2 29.3 29.3 29.4 29.3 29.  28.3 27.4 28.  28.7 28.6
  28.4 28.5 28.5 28.4 28.9 28.7 28.8 28.6 29.  28.6 28.7 28.7 29.1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Moving Average Clossing
Buy if MA(n) > MA(m) for n = 20 and m = 50

In [ ]:
done = False
obs = env.reset() # reset environment move to the first ate
cnt = 0
actions = [0] # first action no hold
status = "No Position"
p,q= pq # ARMA orders
arch_p, arch_q = garch_pq # GARCH orders
alpha = 0.05 # significance level at 5%
action = 0
while not done: # loop until all data are exhausted
  # build trading rule here
  action = 0 # action can be 0 for sell and 1 for buy
  actions.append(action) # keep all action
  obs, reward, done, info = env.step(action) # perform trading
  print(info, portfolio.balances)
portfolio.ledger.as_frame().head(7)

## Determine the trade perfomance

In [ ]:
df = pd.DataFrame(portfolio.performance) # convert profolio performance to dataframe
df = df.T
plt.subplot(2,1,1)
plt.plot(df.net_worth) # plot networths
plt.title("Level Crossing")
plt.subplot(2,1,2)
plt.plot(actions) # plot actions

In [ ]:
print(f"Level crossing Wealth {df.net_worth.iloc[-1]:,.2f}") # last net worth
print(f"Level crossing Gain {df.net_worth.iloc[-1]/100_000:,.2f}") # last net worth
ret = df.net_worth.pct_change(1).dropna()
sharp_ration = ret.mean() / ret.std() #compute sharpe ratio
print(f"Sharpe ratio: {sharp_ration *np.sqrt(365):0.3f}")

# High Low Envelop
Buy if today price is higher than HH(30) and sell if lower than LL(30)

In [ ]:
cash = Wallet(set_market, 100_000 * THB) # initialize at 100k THB
asset = Wallet(set_market, 0 * AOT)
portfolio = Portfolio(THB, [cash,
                            asset])
env = default.create(
        portfolio=portfolio, # start with our portfolio
        action_scheme=default.actions.BSH(cash=cash, asset=asset), # all in and out
        # I made some changes here
        reward_scheme=default.rewards.SimpleProfit(), # simple profit (to be used later)
        feed=feed, # data feed for historical price
        window_size=1, # historical data that we need
        min_periods=hist_len # starting trading location
        )

In [ ]:
done = False
obs = env.reset() # reset environment move to the first ate
cnt = 0
actions = [0] # first action no hold
status = "No Position"
p,q= pq # ARMA orders
arch_p, arch_q = garch_pq # GARCH orders
alpha = 0.05 # significance level at 5%
action = 0
while not done: # loop until all data are exhausted
  # build trading rule here
  action = 0 # action can be 0 for sell and 1 for buy
  actions.append(action) # keep all action
  obs, reward, done, info = env.step(action) # perform trading
  print(info, portfolio.balances)
portfolio.ledger.as_frame().head(7)

In [ ]:
df2 = pd.DataFrame(portfolio.performance) # convert profolio performance to dataframe
df2 = df2.T
plt.subplot(2,1,1)
plt.plot(df2.net_worth) # plot networths
plt.title("High Low")
plt.subplot(2,1,2)
plt.plot(actions) # plot actions

In [ ]:
print(f"High-Lower Envelop Wealth {df2.net_worth.iloc[-1]:,.2f}") # last net worth
print(f"High-Lower Envelop Gain {df2.net_worth.iloc[-1]/100_000:,.2f}") # last net worth
ret = df2.net_worth.pct_change(1).dropna()
sharp_ration = ret.mean() / ret.std() #compute sharpe ratio
print(f"Sharpe ratio: {sharp_ration *np.sqrt(365):0.3f}")

# Trend lines crossing
Let $T_t^u$ be upper trend line and $T^l_t$ be a lower trend line
Buy if $P_t > T_t^u$ and sell if $P_t < T_t^l$


In [ ]:
cash = Wallet(set_market, 100_000 * THB) # initialize at 100k THB
asset = Wallet(set_market, 0 * AOT)
portfolio = Portfolio(THB, [cash,
                            asset])
env = default.create(
        portfolio=portfolio, # start with our portfolio
        action_scheme=default.actions.BSH(cash=cash, asset=asset), # all in and out
        # I made some changes here
        reward_scheme=default.rewards.SimpleProfit(), # simple profit (to be used later)
        feed=feed, # data feed for historical price
        window_size=1, # historical data that we need
        min_periods=hist_len # starting trading location
        )

In [ ]:
done = False
obs = env.reset() # reset environment move to the first ate
cnt = 0
actions = [0] # first action no hold
status = "No Position"
p,q= pq # ARMA orders
arch_p, arch_q = garch_pq # GARCH orders
alpha = 0.05 # significance level at 5%
action = 0
while not done: # loop until all data are exhausted
  # build trading rule here
  action = 0 # action can be 0 for sell and 1 for buy
  actions.append(action) # keep all action
  obs, reward, done, info = env.step(action) # perform trading
  print(info, portfolio.balances)
portfolio.ledger.as_frame().head(7)

In [ ]:
df3 = pd.DataFrame(portfolio.performance) # convert profolio performance to dataframe
df3 = df3.T
plt.subplot(2,1,1)
plt.plot(df3.net_worth) # plot networths
plt.title("Trend Line Crossing")
plt.subplot(2,1,2)
plt.plot(actions) # plot actions

In [ ]:
print(f"Trend Line Crossing Wealth {df2.net_worth.iloc[-1]:,.2f}") # last net worth
print(f"Trend Line Crossing Gain {df2.net_worth.iloc[-1]/100_000:,.2f}") # last net worth
ret = df2.net_worth.pct_change(1).dropna()
sharp_ration = ret.mean() / ret.std() #compute sharpe ratio
print(f"Sharpe ratio: {sharp_ration *np.sqrt(365):0.3f}")